## Imports

In [1]:
#standard useful libraries
import pandas as pd
import numpy as np
import random

#model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#neural network dependencies
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Data

In [2]:
#Loading data from csv
pipes_df = pd.read_csv("Pipe_data.csv")
pipes_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FID,MOID,DIAMETER,PIPE_LENGTH,MATERIAL,DESCRIPTION,YEAR_INSTALLED,PRESSURE_ZONE,LINING,LINE_YEAR,...,WORK_ORDER_NUMBER,Comments,ReportDate,StartDateT,CompletedD,ReportedY,SrvcRqst_1,MATERIAL_1,MUSYM,FAILURE
0,16420,P35881,8.0,861.090481,M.J.,Distribution,1952,Hemp 1175,,0,...,1143689,PER UNIT 819\n\rREQUEST 1 6X12 PLATE AND 4 BAG...,2014-10-25,2014-11-16,2014-11-16,2014,Water Request,CAS,Ub,1
1,44465,P88079,8.0,749.511184,TYT,Distribution,1965,Hemp 1175,,0,...,1311148,MAIN BREAK,2015-08-18,2015-08-18,2015-08-18,2015,Water Request,DIP,ReD,1
2,23810,P53764,8.0,171.554040,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
3,52630,P400741,8.0,42.761097,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
4,10510,P20210,12.0,31.300481,DIP,Transmission,2006,Hemp 1175,,0,...,1628288,PER J. SKINNER MAIN BREAK AT LOCATION\nCall ta...,2017-01-22,2017-02-28,2017-02-28,2017,Turn off burst pipe,DIP,Ub,1


In [3]:
#get dummies for columns where applicable--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['MATERIAL'], prefix='MATERIAL')], axis=1)
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['DESCRIPTION'], prefix='DESCRIPTION')], axis=1); 
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['PRESSURE_ZONE'], prefix='P-ZONE')], axis=1);

#drop original columns--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df2 = pipes_df.drop(columns={ 'MATERIAL',
 'DESCRIPTION',
 'PRESSURE_ZONE',})

#Condense correlating PATTERN + DEMAND columns into 1 column
pipes_df2 = pipes_df2.rename(columns={"DEMAND1":"Demand_RES", "DEMAND2":"Demand_NON-RES","DEMAND3":"Demand_WHOLE","DEMAND4":"Demand_FLUSH","DEMAND5":"Demand_UFW",
                                     "DEMAND6":"Demand_RES-DISTR","DEMAND7":"Demand_NON RES-DISTR","DEMAND8":"Demand_2030-CWTP-NS"})

#Delete unneeded columns
#NOTE to Team: I didn’t see any data at all in the ‘DEMAND9’, ‘PATTERN9’, ‘DEMAND10’, ‘PATTERN10’ columns,so I deleted all of these columns
pipes_df3 = pipes_df2.drop(columns={"PATTERN1", "PATTERN2", "PATTERN3", "PATTERN4", "PATTERN5", "PATTERN6", "PATTERN7", "PATTERN8", "PATTERN9", "PATTERN10", "DEMAND9", "DEMAND10"})

In [4]:
#Create new Age column
#Age is being calculated as 2018 - installation year if no report was made
#Report year - installation year otherwise
tempAgeList = []
for row in np.arange(len(pipes_df3['FID'])):
    try:
        value = int(pipes_df3.iloc[row]['ReportDate'].year) - pipes_df3.iloc[row]['YEAR_INSTALLED']
    except:
        value = 2018 - pipes_df3.iloc[row]['YEAR_INSTALLED']
    tempAgeList.append(value)
pipes_df3['Age']=tempAgeList

In [5]:
#Add a column to be opposite of FAILURE column for use in neural network training
tempSuccessList = []
for row in np.arange(len(pipes_df3['FID'])):
    value = 1 - pipes_df3.iloc[row]['FAILURE']
    tempSuccessList.append(value)
pipes_df3['SUCCESS'] = tempSuccessList

# Machine Learning

# PreProcessing

In [6]:
#Select which columns to train off of.
#A number of iterations of this were initially used when exploring the data
#In the end it was decided to only use average values for those which have averages

inputColumns = ['DIAMETER', 'PIPE_LENGTH','ROUGHNESS',
       'HL1000','AVE_FLOW','AVE_VELOCITY','MAX_VELOCITY','MIN_VELOCITY',
       'AVE_HEADLOSS', 'ELEVATION','RUN_ELEV', 'DEMAND',
       'AVE_HEAD',
       'AVE_PRESS','MATERIAL_AMTYT',
       'MATERIAL_CI', 'MATERIAL_CO', 'MATERIAL_COP', 'MATERIAL_CPP',
       'MATERIAL_Copper', 'MATERIAL_DEL', 'MATERIAL_DIP', 'MATERIAL_EARGEO',
       'MATERIAL_GP', 'MATERIAL_HDPE', 'MATERIAL_L.J.', 'MATERIAL_M.J.',
       'MATERIAL_MCW', 'MATERIAL_OTH', 'MATERIAL_PE', 'MATERIAL_PP',
       'MATERIAL_PVC', 'MATERIAL_R.L.J.', 'MATERIAL_RCP', 'MATERIAL_SP',
       'MATERIAL_ST', 'MATERIAL_T.D', 'MATERIAL_T.D.', 'MATERIAL_TTE',
       'MATERIAL_TYT', 'MATERIAL_UNK', 'MATERIAL_W.I.','Age']

In [7]:
#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pipes_df3[inputColumns],pipes_df3['FAILURE'],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

#Make another split for y which uses the additional 'SUCCESS' column for use in neural network
#Note: Using random state the X_train and X_test would be identical and so need not be kept
_, _, y_train_2, y_test_2 = train_test_split(pipes_df3[inputColumns],pipes_df3[['FAILURE','SUCCESS']],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

In [8]:
#Note: If we were to train and test on the split as it currently is, it will just guess that everything is an unbroken pipe
#This is not desired behavior

#Split X_train into a list of those where the pipes are broken and those which arent
#Then, undersample those pipes which are unbroken to avoid bias
X_train_false = X_train[y_train==0].sample(n=sum(y_train))
X_train_true = X_train[y_train==1].sample(n=sum(y_train))

#Recombine the sampled X_train into one list and keep only those y_train values corresponding to selected indices
X_train_sampled = X_train_true.append(X_train_false)
y_train_sampled = y_train.loc[X_train_sampled.index]
y_train_sampled_2 = y_train_2.loc[X_train_sampled.index]

#Apply min-max scalers to X
#Note: applying min-max too early changed format of dataframe into a np.array and interfered with the sampling process
X_minmax = MinMaxScaler().fit(X_train)
X_train_minmax = X_minmax.transform(X_train_sampled)
X_test_minmax = X_minmax.transform(X_test)

## Logistic Regression

In [9]:
#Set a logistic regression as a model
lrmodel = LogisticRegression()
lrmodel.fit(X_train_minmax,y_train_sampled)
lrmodel.score(X_train_minmax,y_train_sampled)

0.6698073701842546

In [10]:
lrmodel.score(X_test_minmax,y_test)

0.6768626517495834

In [11]:
confusion_matrix(y_test,lrmodel.predict(X_test_minmax))

array([[10711,  5069],
       [  361,   663]], dtype=int64)

## Neural Network

In [12]:
#Add a shallow neural network
nnmodel = Sequential()

#Add layers
nnmodel.add(Dense(100, activation='relu', input_dim=X_train_minmax.shape[1]))
nnmodel.add(Dense(2,activation = 'softmax'))

#Compile
nnmodel.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

#Fit the training model to the data
nnmodel.fit(
    X_train_minmax,
    y_train_sampled_2,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 0s - loss: 0.6681 - acc: 0.5827
Epoch 2/100
 - 0s - loss: 0.6431 - acc: 0.6237
Epoch 3/100
 - 0s - loss: 0.6328 - acc: 0.6390
Epoch 4/100
 - 0s - loss: 0.6245 - acc: 0.6552
Epoch 5/100
 - 0s - loss: 0.6163 - acc: 0.6656
Epoch 6/100
 - 0s - loss: 0.6103 - acc: 0.6669
Epoch 7/100
 - 0s - loss: 0.6021 - acc: 0.6736
Epoch 8/100
 - 0s - loss: 0.5958 - acc: 0.6815
Epoch 9/100
 - 0s - loss: 0.5932 - acc: 0.6853
Epoch 10/100
 - 0s - loss: 0.5892 - acc: 0.6889
Epoch 11/100
 - 0s - loss: 0.5868 - acc: 0.6910
Epoch 12/100
 - 0s - loss: 0.5862 - acc: 0.6928
Epoch 13/100
 - 0s - loss: 0.5823 - acc: 0.6958
Epoch 14/100
 - 0s - loss: 0.5825 - acc: 0.6972
Epoch 15/100
 - 0s - loss: 0.5807 - acc: 0.7035
Epoch 16/100
 - 0s - loss: 0.5786 - acc: 0.7023
Epoch 17/100
 - 0s - loss: 0.5780 - acc: 0.6983
Epoch 18/100
 - 0s - loss: 0.5778 - acc: 0.7002
Epoch 19/100
 - 0s - loss: 0.5760 - acc: 0.6993
Epoch 20/100
 - 0s - loss: 0.5755 - acc: 0.7023
Epoch 21/100
 - 0s - loss: 0.5744 - acc: 0.7008
E

In [13]:
nnmodel_loss, nnmodel_accuracy = nnmodel.evaluate(X_test_minmax, y_test_2, verbose=2)
print(f"Loss: {nnmodel_loss}, Accuracy: {nnmodel_accuracy}")

Loss: 0.7346381294928798, Accuracy: 0.6005117829088312


## Random Forests

### Decide on number of estimators to use

In [14]:
#Load a Random Forest model
model = RandomForestClassifier()

#Set a parameter grid
#Only real parameters we needed to check was the number of estimators
param_grid = {'n_estimators': [20,50,100,200,300]}
grid = GridSearchCV(model, param_grid, verbose=3)

#Fit the grid
grid.fit(X_train_minmax,y_train_sampled)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=20 .................................................
[CV] ........ n_estimators=20, score=0.7380653266331658, total=   0.0s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ........ n_estimators=20, score=0.7405778894472361, total=   0.0s
[CV] n_estimators=20 .................................................
[CV] ........ n_estimators=20, score=0.7198492462311558, total=   0.0s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7562814070351759, total=   0.2s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7556532663316583, total=   0.2s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7430904522613065, total=   0.2s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7487437185929648, total=   0.7s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7650753768844221, total=   0.5s
[CV] n_estimators=100 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   14.3s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [15]:
#Find the best score and best parameters
print(f'The best score was {grid.score(X_test_minmax,y_test)}\nThe best parameters were {grid.best_params_}')

The best score was 0.7413711021185432
The best parameters were {'n_estimators': 200}


In [16]:
#Take a look at classification report to confirm that the model is ideal
print(classification_report(y_test, grid.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       0.98      0.74      0.84     15780
          1       0.16      0.75      0.26      1024

avg / total       0.93      0.74      0.81     16804



In [17]:
#Take a look at the confusion matrix to confirm that there is no unintended behavior (e.g. only guessing unbroken)
confusion_matrix(y_test, grid.predict(X_test_minmax))

array([[11690,  4090],
       [  256,   768]], dtype=int64)

# Performing deeper analysis with Random Forests

In [18]:
#Apply cross validation so as to have percentage predictions for all data points without bias towards training data
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
split = cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE'])
rfmodel = RandomForestClassifier(n_estimators=300)

In [19]:
#Create new column for use in displaying results of analysis to be filled in later
#Final results will be stored in final_df
scores = []
final_df = pipes_df3
final_df['BurstChance'] = 0

In [20]:
#For each train-test split from our crossvalidation, run the same type of analysis as before
for train_idx, test_idx in cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE']):
    Current_X_train = pipes_df3[inputColumns].loc[train_idx]
    Current_X_test = pipes_df3[inputColumns].loc[test_idx]
    Current_y_train = pipes_df3['FAILURE'].loc[train_idx]
    Current_y_test = pipes_df3['FAILURE'].loc[test_idx]
    
    
    #Undersample training data as before
    Current_X_train_false = Current_X_train[Current_y_train==0].sample(n=sum(Current_y_train))
    Current_X_train_true = Current_X_train[Current_y_train==1].sample(n=sum(Current_y_train))
    
    Current_X_train_sampled = Current_X_train_true.append(Current_X_train_false)
    Current_y_train_sampled = Current_y_train.loc[Current_X_train_sampled.index]
    
    Current_X_minmax = MinMaxScaler().fit(Current_X_train)
    Current_X_train_minmax = Current_X_minmax.transform(Current_X_train_sampled)
    Current_X_test_minmax = Current_X_minmax.transform(Current_X_test)

    
    rfmodel.fit(Current_X_train_minmax, Current_y_train_sampled)
    score = rfmodel.score(Current_X_test_minmax, Current_y_test)
    scores.append(score)
    print(confusion_matrix(Current_y_test,rfmodel.predict(Current_X_test_minmax)))
    
    #Store prediction probabilities into final_df
    final_df['BurstChance'].loc[test_idx] = [x[1] for x in list(rfmodel.predict_proba(Current_X_test_minmax))]


[[7839 2681]
 [ 134  549]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8053 2467]
 [ 162  521]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[7961 2559]
 [ 161  521]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[7817 2703]
 [ 152  530]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[7909 2611]
 [ 163  519]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [21]:
#Look at final scores for each
scores

[0.7487280192805499,
 0.7653307149870571,
 0.7571862167470095,
 0.7451347973576147,
 0.7523656489912516]

In [22]:
#Output top ten most likely pipes to burst which are currently unbroken according to our model
final_df[['BurstChance','FID','MOID'] + inputColumns].loc[final_df['FAILURE']==0].sort_values('BurstChance', ascending=False).head(10)

,BurstChance,FID,MOID,DIAMETER,PIPE_LENGTH,ROUGHNESS,HL1000,AVE_FLOW,AVE_VELOCITY,MAX_VELOCITY,...,MATERIAL_RCP,MATERIAL_SP,MATERIAL_ST,MATERIAL_T.D,MATERIAL_T.D.,MATERIAL_TTE,MATERIAL_TYT,MATERIAL_UNK,MATERIAL_W.I.,Age
1913,0.986667,10000,P29568,6.0,536.392085,39,0.1814,18.4316,0.2091,0.2698,...,0,0,0,0,0,0,0,0,0,82
937,0.980000,33143,P6285,42.0,444.844718,130,0.0001,173.7656,0.0402,0.0560,...,0,0,0,0,0,0,0,0,0,60
8218,0.980000,11319,P2446,36.0,57.899811,152,0.1054,4053.0063,1.2775,1.6978,...,0,0,0,0,0,0,0,0,0,20
919,0.976667,954,P103643,42.0,855.946881,130,0.0000,173.7656,0.0402,0.0560,...,0,0,0,0,0,0,0,0,0,60
29050,0.973333,29112,P46120,6.0,583.595496,84,0.0013,1.8730,0.0213,0.0251,...,0,0,0,0,0,0,0,0,0,61
928,0.973333,30911,P64867,42.0,713.707471,130,0.0003,135.6322,0.0314,0.0835,...,0,0,0,0,0,0,0,0,0,60
941,0.973333,53174,P161395,42.0,1129.185635,130,0.0000,0.0000,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0,60
2146,0.973333,16996,P38945,6.0,651.419297,39,0.0630,16.1656,0.1834,0.2739,...,0,0,0,0,0,0,0,0,0,78
3783,0.966667,52675,P401104,8.0,498.518309,133,0.1499,89.7236,0.5727,0.7748,...,0,0,0,0,0,0,0,0,0,2
22163,0.966667,17116,P36285,6.0,609.982437,84,0.0008,1.2515,0.0142,0.0161,...,0,0,0,0,0,0,0,0,0,66


In [23]:
#Output most relevant parameters according to our recent model
print(sorted(zip(map(lambda x: round(x, 4), rfmodel.feature_importances_), inputColumns), reverse=True))

[(0.1273, 'PIPE_LENGTH'), (0.0799, 'AVE_HEAD'), (0.0732, 'DEMAND'), (0.0713, 'Age'), (0.0665, 'AVE_HEADLOSS'), (0.0623, 'AVE_PRESS'), (0.0593, 'AVE_FLOW'), (0.0583, 'MIN_VELOCITY'), (0.058, 'MAX_VELOCITY'), (0.0577, 'RUN_ELEV'), (0.0566, 'ELEVATION'), (0.0564, 'AVE_VELOCITY'), (0.053, 'HL1000'), (0.048, 'ROUGHNESS'), (0.0354, 'DIAMETER'), (0.0077, 'MATERIAL_T.D.'), (0.0059, 'MATERIAL_DIP'), (0.0056, 'MATERIAL_M.J.'), (0.0045, 'MATERIAL_TYT'), (0.0033, 'MATERIAL_DEL'), (0.0032, 'MATERIAL_L.J.'), (0.0022, 'MATERIAL_UNK'), (0.0013, 'MATERIAL_W.I.'), (0.0008, 'MATERIAL_CPP'), (0.0006, 'MATERIAL_CO'), (0.0006, 'MATERIAL_CI'), (0.0004, 'MATERIAL_ST'), (0.0003, 'MATERIAL_OTH'), (0.0, 'MATERIAL_TTE'), (0.0, 'MATERIAL_T.D'), (0.0, 'MATERIAL_SP'), (0.0, 'MATERIAL_RCP'), (0.0, 'MATERIAL_R.L.J.'), (0.0, 'MATERIAL_PVC'), (0.0, 'MATERIAL_PP'), (0.0, 'MATERIAL_PE'), (0.0, 'MATERIAL_MCW'), (0.0, 'MATERIAL_HDPE'), (0.0, 'MATERIAL_GP'), (0.0, 'MATERIAL_EARGEO'), (0.0, 'MATERIAL_Copper'), (0.0, 'MATERIAL